In [1]:
import random
import numpy as np
import time
import math
import uuid
import csv
import os
from math import cos, exp, sqrt, pi

In [2]:
POP_SIZE = 100
N_EVALS = 100000
N_REPLICATES = 30
MIN_LEN = 3
MAX_LEN = 100
gene_low, gene_high = -2.0, 2.0

F_INIT = 0.8
CR_INIT = 0.9
P_LEN_CHANGE = 0.05

time_str = time.strftime('%Y%m%d_%H%M%S')
out_dir = f'DE_VarialbeDimensions_results_{time_str}'
os.makedirs(out_dir, exist_ok=True)
csv_path = os.path.join(out_dir, 'DE_VariableDimensions_results.csv')

In [3]:
def mkPenalizer(d_target, lam=10, deadzone=2):
    def _pen(ind):
        dev = abs(len(ind) - d_target)
        return 0 if dev <= deadzone else lam * dev
    return _pen

def loadSantaFeMap(path='santafe_trail.txt'):
    world_rows = []
    start_x = start_y = None
    with open(path) as f:
        for y,line in enumerate(f):
            row=[]
            for x,c in enumerate(line.rstrip("\n")):
                if c=='#': row.append(1)
                elif c in ('.','S'):
                    row.append(0)
                    if c=='S': start_x,start_y = x,y
            if row: world_rows.append(row)
    if not world_rows: raise ValueError(f'No valid rows in map: {path}')
    H=len(world_rows); W=len(world_rows[0])
    for r in world_rows:
        if len(r)!=W: raise ValueError('Inconsistent row lengths.')
    world=[c for row in world_rows for c in row]
    if start_x is None or start_y is None: start_x=start_y=0
    return world, W, H, start_x, start_y

world, W, H, START_X, START_Y = loadSantaFeMap()

In [4]:
def sphere(d_target, lam):
    pen = mkPenalizer(d_target, lam)
    return lambda ind: sum(x*x for x in ind) + pen(ind)

def rastrigin(d_target, lam):
    pen = mkPenalizer(d_target, lam)
    return lambda ind: 10*len(ind) + sum(x*x-10*cos(2*pi*x) for x in ind) + pen(ind)

def griewank(d_target, lam):
    pen = mkPenalizer(d_target, lam)
    def f(ind):
        s = sum(x*x for x in ind)
        p = np.prod([cos(x/sqrt(i+1)) for i,x in enumerate(ind)])
        return 1 + s/4000. - p + pen(ind)
    return f

def ackley(d_target, lam):
    pen = mkPenalizer(d_target, lam)
    def f(ind):
        d = len(ind)
        s = sum(x*x for x in ind)
        c = sum(cos(2*pi*x) for x in ind)
        return -20*exp(-0.2*sqrt(s/d)) - exp(c/d) + 20 + math.e + pen(ind)
    return f

def koza(d_target, lam):
    pen = mkPenalizer(d_target, lam)
    xs = np.linspace(-1,1,20)
    def f(ind):
        err=0
        for x in xs:
            t = x**4 + x**3 + x**2 + x
            p = sum(w*(x**i) for i,w in enumerate(ind))
            err += (p-t)**2
        return err + pen(ind)
    return f

def knapsack(d_target, lam, items, cap):
    pen = mkPenalizer(d_target, lam)
    def f(ind):
        sel=[int(round(x))%2 for x in ind[:len(items)]]
        w_sum=sum(w for (w,v),s in zip(items,sel) if s)
        v_sum=sum(v for (w,v),s in zip(items,sel) if s)
        over=max(0,w_sum-cap)
        return -v_sum + 1000*over + pen(ind)
    return f

def parity(N, lam):
    d_target=2**N
    pen=mkPenalizer(d_target,lam)
    def f(ind):
        table=[int(round(x))%2 for x in ind[:d_target]]
        errs=0
        for i in range(d_target):
            target = bin(i).count('1')%2
            if i < len(table):
                if table[i]!=target: errs+=1
            else:
                errs+=1
        return errs + pen(ind)
    return f

def santaFe(d_target, lam):
    pen = mkPenalizer(d_target, lam)
    def f(ind, max_steps=400):
        dir=0; x,y=START_X,START_Y; food=0
        visited = world.copy()
        idx=lambda xx,yy: (yy%H)*W + (xx%W)
        for i in range(min(len(ind), max_steps)):
            if visited[idx(x,y)]==1:
                food+=1; visited[idx(x,y)]=0
            act = int(abs(ind[i]))%3
            if act==0:
                dx,dy=[(0,1),(1,0),(0,-1),(-1,0)][dir]; x+=dx; y+=dy
            elif act==1:
                dir=(dir+1)%4
            else:
                dir=(dir-1)%4
        return -food + pen(ind)
    return f

In [5]:
def generateInd():
    length = random.randint(MIN_LEN, MAX_LEN)
    return [random.uniform(gene_low, gene_high) for _ in range(length)]

def initPop():
    return [generateInd() for _ in range(POP_SIZE)]

In [6]:
def DEMutate(pop, j, F):
    idxs=list(range(len(pop)))
    idxs.remove(j)
    a,b,c = random.sample(idxs,3)
    va, vb, vc = pop[a], pop[b], pop[c]
    d_mut = min(len(va), len(vb), len(vc))
    mutant = [va[i] + F*(vb[i] - vc[i]) for i in range(d_mut)]
    if random.random() < P_LEN_CHANGE:
        if random.random() < 0.5 and len(mutant) < MAX_LEN:
            mutant.append(random.uniform(gene_low, gene_high))
        elif len(mutant) > MIN_LEN:
            mutant.pop(random.randrange(len(mutant)))
    while len(mutant) < MIN_LEN:
        mutant.append(random.uniform(gene_low, gene_high))
    return mutant[:MAX_LEN]

def DECrossover(target, mutant, CR):
    dt, dm = len(target), len(mutant)
    L = max(dt, dm)
    j_rand = random.randrange(L)
    trial = []
    for i in range(L):
        if random.random() < CR or i == j_rand:
            if i < dm:
                trial.append(mutant[i])
            else:
                trial.append(random.uniform(gene_low, gene_high))
        else:
            if i < dt:
                trial.append(target[i])
            else:
                trial.append(random.uniform(gene_low, gene_high))
    while len(trial) < MIN_LEN:
        trial.append(random.uniform(gene_low, gene_high))
    return trial[:MAX_LEN]

In [7]:
def run_de(fit_func, name):
    pop = initPop()
    evals = 0
    best_hist=[]
    while evals < N_EVALS:
        gen_idx = len(best_hist)
        F = F_INIT * (1 - gen_idx/(N_EVALS/POP_SIZE))
        CR = CR_INIT * (1 - 0.5*gen_idx/(N_EVALS/POP_SIZE))
        new_pop=[]
        for j,x in enumerate(pop):
            mutant = DEMutate(pop, j, F)
            trial = DECrossover(x, mutant, CR)
            if fit_func(trial) <= fit_func(x):
                new_pop.append(trial)
            else:
                new_pop.append(x)
        pop = new_pop
        evals += POP_SIZE
        best = fit_func(min(pop, key=fit_func))
        best_hist.append(best)
        if gen_idx % 10 == 0:
            elite = min(pop, key=fit_func)
            print(f'{name} Evals {evals}: Best = {best:.6f}, Len = {len(elite)}')
    return min(pop, key=fit_func), best_hist

In [8]:
with open(csv_path, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile,
                            fieldnames=['run_id','benchmark','replicate','seed','best','len_final','cpu_s'])
    writer.writeheader()

    items = [(random.randint(1,20), random.randint(10,100)) for _ in range(20)]
    benchmarks = {
        'Sphere': sphere(30,10),
        'Rastrigin': rastrigin(30,10),
        'Griewank': griewank(30,10),
        'Ackley': ackley(30,10),
        'Koza': koza(10,10),
        'Knapsack': knapsack(20,0,items,50),
        'Parity': parity(6,10),
        'SantaFe': santaFe(100,10)
    }

    for name, func in benchmarks.items():
        for rep in range(N_REPLICATES):
            seed = random.randint(0,2**31-1)
            random.seed(seed); np.random.seed(seed)
            start = time.perf_counter()
            best_ind, hist = run_de(func, name)
            cpu = time.perf_counter() - start
            writer.writerow({
                'run_id': uuid.uuid4(),
                'benchmark': name,
                'replicate': rep,
                'seed': seed,
                'best': round(func(best_ind),6),
                'len_final': len(best_ind),
                'cpu_s': round(cpu,4)
            })
print(f'Results written to {csv_path}')

Sphere Evals 100: Best = 40.531684, Len = 30
Sphere Evals 1100: Best = 40.531684, Len = 30
Sphere Evals 2100: Best = 40.531684, Len = 30
Sphere Evals 3100: Best = 36.378016, Len = 28
Sphere Evals 4100: Best = 36.378016, Len = 28
Sphere Evals 5100: Best = 36.378016, Len = 28
Sphere Evals 6100: Best = 36.378016, Len = 28
Sphere Evals 7100: Best = 36.378016, Len = 28
Sphere Evals 8100: Best = 35.606977, Len = 30
Sphere Evals 9100: Best = 28.610593, Len = 31
Sphere Evals 10100: Best = 28.610593, Len = 31
Sphere Evals 11100: Best = 28.610593, Len = 31
Sphere Evals 12100: Best = 27.157937, Len = 31
Sphere Evals 13100: Best = 25.722112, Len = 30
Sphere Evals 14100: Best = 22.479062, Len = 31
Sphere Evals 15100: Best = 22.479062, Len = 31
Sphere Evals 16100: Best = 22.479062, Len = 31
Sphere Evals 17100: Best = 22.479062, Len = 31
Sphere Evals 18100: Best = 22.159311, Len = 30
Sphere Evals 19100: Best = 22.159311, Len = 30
Sphere Evals 20100: Best = 22.159311, Len = 30
Sphere Evals 21100: Best